In [2]:
#%% import
import netCDF4 as nc
from typing import Optional
import numpy as np
import pandas as pd
import configparser
from sqlalchemy import create_engine, text
import psycopg2
from pathlib import Path
import os
import sys
# import writeMeteoModel2db
from writeMeteoModel2db import get_engine, write_centroids_in_db, add_geom_column,find_entrys_wo_weatherdata,find_places_from_meta,add_index_weatherdata,create_table_weatherdata,find_missing_years_and_fill,get_weatherdata_from_model,get_coords_weathermodel,write_weatherdata_into_db,mark_ts_entries_with_wm_index


# import energyants
sys.path.append(r'C:\Users\agoldmaier\Documents\GitLabFhG\energyants')
from energyants.meteo.models.era5land.model import Era5Land 
from energyants.meteo.models.sarah3.model import SARAH3

In [8]:
print('start')
## Variables snh data4grid
db_name_data = 'last_secure'
db_name_weather = 'last'
dataset_ts = 'smartmeter_data4grid_snh'
dataset_meta = 'smartmeter_data4grid_snh_meta'
dataset_id = 'id'
dataset_time_column = 'time'
dataset_ix_wm = 'id_era5_land'
table_weathermodel = 'era5_land_centroid_deu_landpixel'
table_wm_index = 'id_era5_land'
tablename_weatherdata = 'era5_land'
name_geom_column = 'geom'

start


In [ ]:
print('start')
## Variables snh
db_name_data = 'last_secure'
db_name_weather = 'last'
dataset_ts = 'smartmeter_snh'
dataset_meta = 'smartmeter_snh_meta'
dataset_id = 'id'
dataset_time_column = 'time'
dataset_ix_wm = 'id_era5_land'
table_weathermodel = 'era5_land_centroid_deu_landpixel'
table_wm_index = 'id_era5_land'
tablename_weatherdata = 'era5_land'
name_geom_column = 'geom'


In [2]:
print('start')
## Variables openmeter
db_name_data = 'last'
db_name_weather = 'last'
dataset_ts = 'openmeter_measurement_ts'
dataset_meta = 'openmeter_meta_mit_erz'
dataset_id = 'sensor_id'
dataset_time_column = 'start_time'
dataset_ix_wm = 'id_era5_land'
table_weathermodel = 'era5_land_centroid_deu_landpixel'
table_wm_index = 'id_era5_land'
tablename_weatherdata = 'era5_land'
name_geom_column = 'geom_plz'

start


In [ ]:
print('start')
## Variables wpuq
db_name_data = 'last'
db_name_weather = 'last'
dataset_ts = 'wpuq_ts'
dataset_meta = 'wpuq_meta'
dataset_id = 'id'
dataset_time_column = 'time'
dataset_ix_wm = 'id_era5_land'
table_weathermodel = 'era5_land_centroid_deu_landpixel'
table_wm_index = 'id_era5_land'
tablename_weatherdata = 'era5_land'
name_geom_column = 'geom'

In [ ]:
print('start')
## Variables wpuq
db_name_data = 'last'
db_name_weather = 'last'
dataset_ts = 'wpuq_ts'
dataset_meta = 'wpuq_meta'
dataset_id = 'id'
dataset_time_column = 'time'
dataset_ix_wm = 'id_era5_land'
table_weathermodel = 'era5_land_centroid_deu_landpixel'
table_wm_index = 'id_era5_land'
tablename_weatherdata = 'era5_land'
name_geom_column = 'geom'

In [4]:
# Verbindung DB Lastdaten
engine_data = get_engine(db_name_data)

# Verbindung DB Lastdaten
if db_name_weather == db_name_data:
        engine_weather = engine_data
else:
        engine_weather = get_engine(db_name_weather)

In [ ]:
## 0.1 Lookuptable der Wettermodellcentroide in die Datenbank schreiben
# filepath=r"C:\Users\agoldmaier\Documents\meteoData\ERA5_Land_grid.nc"
# write_centroids_in_db(filepath,table_weathermodel,engine_data)
# print('0.1 done')

In [ ]:
## 0.2 Geometriespalte in Metadata hinzufügen
# add_geom_column(dataset_meta,engine_data)
# print('0.2 done')
# name_geom_column = 'geom_plz'

In [9]:
## 1. Wetterdatenindex zu Metadaten hinzufügen, wenn noch nicht vorhanden
add_index_weatherdata(dataset_meta,dataset_id,dataset_ix_wm,table_weathermodel,table_wm_index,engine_data,column_geom=name_geom_column)
print('1. done')

<class 'pandas.core.frame.DataFrame'>
   id
0  49
1  50
2  51
3  63
4  64

         UPDATE smartmeter_data4grid_snh_meta 
         SET id_era5_land_neu = closest_table2_id
         FROM
            (SELECT  smartmeter_data4grid_snh_meta.id AS table1_id,
                    ( SELECT era5_land_centroid_deu_landpixel.id_era5_land
                        FROM era5_land_centroid_deu_landpixel
                        ORDER BY smartmeter_data4grid_snh_meta.geom <-> era5_land_centroid_deu_landpixel.geom
                        LIMIT 1
                    ) AS closest_table2_id
            FROM (SELECT * 
                  FROM smartmeter_data4grid_snh_meta 
                  WHERE id_era5_land_neu IS NULL
                  ) as smartmeter_data4grid_snh_meta
            ) as dist
         WHERE smartmeter_data4grid_snh_meta.id = dist.table1_id
1. done


In [ ]:
## 2. Tabelle für Wetterdaten erzeugen, wenn noch nicht vorhanden
create_table_weatherdata(tablename_weatherdata,engine_weather)
print('2. done')

In [4]:
## 3. In Zeitreihendaten prüfen, für welche Orte und Zeitstempel noch keine Wetterdaten zugeordnet sind.
entrys_wo_weather = find_entrys_wo_weatherdata(dataset_ts,dataset_meta,dataset_id,dataset_time_column,dataset_ix_wm,engine_data)
print('3. done')
print(entrys_wo_weather.head())


            SELECT 
                data.id as id, min(data.time at time zone 'UTC') as mintime,
                  max(data.time at time zone 'UTC') as maxtime, meta.ix_era5_land as index_wm
            FROM public.smartmeter_data4grid_snh as data, public.smartmeter_data4grid_snh_meta as meta
            WHERE data.ix_era5_land IS NULL AND data.id = meta.id
            GROUP BY data.id, index_wm;
            
3. done
   id             mintime             maxtime  index_wm
0   1 2016-12-31 23:00:00 2021-09-30 21:30:00   1310500
1   2 2016-12-31 23:00:00 2019-08-02 07:00:00   1310500
2   3 2016-12-31 23:00:00 2019-06-24 11:15:00   1310500
3   4 2016-12-31 23:00:00 2021-09-30 21:30:00   1310500
4   5 2016-12-31 23:00:00 2021-09-30 21:30:00   1310500


In [6]:
## 4. In Wetterdatentabelle prüfen, welche der fehlenden Zeiträume vorhanden sind und welche nicht. 
missingYears = find_missing_years_and_fill(entrys_wo_weather,engine_weather)
print('4. done')
print(missingYears)

missingYears_unique = missingYears[['index_wm','Missing Years']].drop_duplicates()


SELECT DISTINCT EXTRACT(year FROM time) AS year FROM era5_land WHERE index = '1317701' AND time >= '2016-12-31 23:00:00' AND time <= '2021-09-15 19:45:00'
result: 
     year
0  2018.0
1  2016.0
2  2021.0
3  2017.0
existing_years: 
[2018.0, 2016.0, 2021.0, 2017.0]
all_years: 
[2016, 2017, 2018, 2019, 2020, 2021]
missing_years: 
[2019, 2020]
missing_years_df_temp: 
   index_wm  Missing Years
0   1317701           2019
1   1317701           2020
SELECT DISTINCT EXTRACT(year FROM time) AS year FROM era5_land WHERE index = '1314099' AND time >= '2016-12-31 23:00:00' AND time <= '2020-03-04 07:30:00'
result: 
     year
0  2018.0
1  2016.0
2  2017.0
existing_years: 
[2018.0, 2016.0, 2017.0]
all_years: 
[2016, 2017, 2018, 2019, 2020]
missing_years: 
[2019, 2020]
missing_years_df_temp: 
   index_wm  Missing Years
0   1314099           2019
1   1314099           2020
SELECT DISTINCT EXTRACT(year FROM time) AS year FROM era5_land WHERE index = '1306902' AND time >= '2016-12-31 23:00:00' AND time 

In [7]:
## 5. Koordinaten der Wettermodelpixel abfragen
coords_wm = get_coords_weathermodel(missingYears_unique['index_wm'],table_weathermodel,engine_weather)
print('5. done')
missingYears_unique = missingYears_unique.merge(coords_wm, left_on='index_wm', right_on='index_wm')

print(missingYears_unique)

5. done
  index_wm Missing Years   lat   lon
0  1317701          2019  53.4  10.1
1  1317701          2020  53.4  10.1
2  1314099          2019  53.5   9.9
3  1314099          2020  53.5   9.9
4  1306902          2019  53.7  10.2
5  1306902          2020  53.7  10.2
6  1306900          2019  53.7  10.0
7  1306900          2020  53.7  10.0
8  1310498          2019  53.6   9.8
9  1310498          2020  53.6   9.8


In [8]:
## 6. Die fehlenden Wetterdaten aus dem Wettermodell abfragen
wd_missing = get_weatherdata_from_model(missingYears_unique)
print('6. done')
print(wd_missing.head())

6. done
                                  t2m  ghi      ws10    index
2019-01-01 00:00:00+00:00  280.119669  0.0  5.400709  1317701
2019-01-01 01:00:00+00:00  280.282822  0.0  5.705382  1317701
2019-01-01 02:00:00+00:00  280.397748  0.0  5.884909  1317701
2019-01-01 03:00:00+00:00  280.477785  0.0  6.026917  1317701
2019-01-01 04:00:00+00:00  280.631704  0.0  6.310211  1317701


In [9]:
## 7. Die Wetterdaten in die Datenbank schreiben
write_weatherdata_into_db(wd_missing,tablename_weatherdata,engine_weather)
print('7. done')

                                  t2m  ghi      ws10    index
2019-01-01 00:00:00+00:00  280.119669  0.0  5.400709  1317701
2019-01-01 01:00:00+00:00  280.282822  0.0  5.705382  1317701
2019-01-01 02:00:00+00:00  280.397748  0.0  5.884909  1317701
2019-01-01 03:00:00+00:00  280.477785  0.0  6.026917  1317701
2019-01-01 04:00:00+00:00  280.631704  0.0  6.310211  1317701
7. done


In [5]:
# Fällt weg wegen Performance (Datenbank stürzt ab, bei der Menge der Anfragen)
## 8. Alle Einträge aus 3. mit index aus Wettermodell versehen, damit klar ist, dass die Wetterdaten vorhanden sind.
# mark_ts_entries_with_wm_index(entrys_wo_weather,dataset_ts,dataset_id,dataset_time_column,engine_data)
# print('8. done')

UPDATE smartmeter_data4grid_snh as data 
                SET ix_era5_land = 1310500
                WHERE data.id = '1' AND time >= '2016-12-31 23:00:00' AND time <= '2021-09-30 21:30:00'
                
UPDATE smartmeter_data4grid_snh as data 
                SET ix_era5_land = 1310500
                WHERE data.id = '2' AND time >= '2016-12-31 23:00:00' AND time <= '2019-08-02 07:00:00'
                
UPDATE smartmeter_data4grid_snh as data 
                SET ix_era5_land = 1310500
                WHERE data.id = '3' AND time >= '2016-12-31 23:00:00' AND time <= '2019-06-24 11:15:00'
                
UPDATE smartmeter_data4grid_snh as data 
                SET ix_era5_land = 1310500
                WHERE data.id = '4' AND time >= '2016-12-31 23:00:00' AND time <= '2021-09-30 21:30:00'
                
UPDATE smartmeter_data4grid_snh as data 
                SET ix_era5_land = 1310500
                WHERE data.id = '5' AND time >= '2016-12-31 23:00:00' AND time <= '2021-09-30 21

In [6]:
# Datenbankverbindung schließen
engine_data.dispose()
engine_weather.dispose()
print('finish')

finish
